In [2]:
# env = myenv

In [3]:
from langchain_openai import OpenAI, ChatOpenAI

In [4]:
from langchain_core.prompts import PromptTemplate

In [5]:
from langchain_classic.chains import LLMChain, SequentialChain

In [6]:
from langchain_community.callbacks import get_openai_callback

In [7]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from pypdf import PdfReader


In [8]:
load_dotenv()

True

In [9]:
key=os.getenv("openai_api_key")

In [10]:
print(key)

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


In [11]:
llm=ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    openai_api_key=key,
    tiktoken_model_name='openai/gpt-oss-20b:free',
    temperature=0.7
    )

In [12]:
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000001DDC0984E20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001DDC1152410>, root_client=<openai.OpenAI object at 0x000001DDC0985060>, root_async_client=<openai.AsyncOpenAI object at 0x000001DDC1152380>, temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1', tiktoken_model_name='openai/gpt-oss-20b:free')

In [13]:
with open(r"C:\Users\user\OneDrive\Desktop\Generative AI\genai-github\response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [14]:
template = """
Text:
{text}

You are an expert MCQ generator.

Your task is to create exactly {number} multiple-choice questions (MCQs) 
for {subject} students using the above text.

Difficulty / Tone:
{tone}

Instructions:
- Each question must be clear and relevant to the given text.
- Do NOT repeat any questions.
- Each MCQ must have four options.
- Only one option should be correct.
- Ensure all questions are conceptually accurate.

Return the output strictly in the JSON format shown below.
Do NOT add any extra text outside the JSON.

### RESPONSE_JSON
{RESPONSE_JSON}
"""


In [15]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','RESPONSE_JSON'],
    template=template
)

In [16]:
quiz_chain=LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key='quiz',
    verbose=True
    )

C:\Users\user\AppData\Local\Temp\ipykernel_12052\1973328359.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(


In [17]:
template2 = """
You are an expert educational content reviewer.

Below is a quiz for {subject} students.

Your task is to:
1. Verify the factual correctness of each MCQ.
2. Confirm that the provided correct answers are accurate.
3. Improve grammar, clarity, and wording where needed.
4. Ensure all questions meet proper academic standards.
5. Fix or upgrade any MCQ that has errors or ambiguity.
6. Do NOT change the total number of questions.
7. Follow the response structure exactly as defined in response.json.

### QUIZ DATA
{quiz}
"""


In [18]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [19]:
review_chain=LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key='review',
    verbose=True
    )

In [20]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text', 'number', 'subject','RESPONSE_JSON', 'tone'],  # combined input vars
    output_variables=['quiz', 'review'],                    # plural
    verbose=True
)

In [21]:
path = r"C:\Users\user\OneDrive\Desktop\Generative AI\genai-github\data.txt"

In [22]:
with open(path,'r') as file:
    Text=file.read()

In [23]:
Text,
number=5,
subject='AI',
tone='simple',
RESPONSE_JSON=RESPONSE_JSON

In [27]:
inputs = {
    "text": Text,
    "number": number,
    "subject": subject,
    "tone": tone,
    "RESPONSE_JSON":json.dumps(RESPONSE_JSON)
}
with get_openai_callback() as cb:
    response = generate_evaluate_chain(inputs)  # Direct call




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
This article is about the pre-1947 history of Pakistan. For post-1946 history, see History of Pakistan (1947–present).
Part of a series on the
History of Pakistan
Arms of Pakistan
Timeline
Ancient
Classical
Medieval
Early modern
Modern
History of provinces
 Category Portal
vte
Part of a series on the
History of South Asia
South Asia (orthographic projection)
Outline
National histories
Regional histories
Specialised histories
vte

A map outlining historical sites in Pakistan.
The history of Pakistan preceding the country's independence in 1947[1] is shared with that of Afghanistan, India, and Iran. Spanning the western expanse of the Indian subcontinent and the eastern borderlands of the Iranian plateau, the region of present-day Pakistan served both as the fertile ground of a major civilization and as the gateway of South Asia to Central Asia and the Near East.[2][3]

Situated o

In [26]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:11124
Prompt Tokens:10459
Completion Tokens:665
Total Cost:0.0


In [28]:
response

{'text': 'This article is about the pre-1947 history of Pakistan. For post-1946 history, see History of Pakistan (1947–present).\nPart of a series on the\nHistory of Pakistan\nArms of Pakistan\nTimeline\nAncient\nClassical\nMedieval\nEarly modern\nModern\nHistory of provinces\n Category Portal\nvte\nPart of a series on the\nHistory of South Asia\nSouth Asia (orthographic projection)\nOutline\nNational histories\nRegional histories\nSpecialised histories\nvte\n\nA map outlining historical sites in Pakistan.\nThe history of Pakistan preceding the country\'s independence in 1947[1] is shared with that of Afghanistan, India, and Iran. Spanning the western expanse of the Indian subcontinent and the eastern borderlands of the Iranian plateau, the region of present-day Pakistan served both as the fertile ground of a major civilization and as the gateway of South Asia to Central Asia and the Near East.[2][3]\n\nSituated on the first coastal migration route of Homo sapiens out of Africa, the re

In [33]:
quiz=response.get("quiz")

In [34]:
quiz=json.loads(quiz)

In [36]:
quiz

{'1': {'no': '1',
  'mcq': 'Which dynasty ruled Sindh and parts of Kutch, Punjab, and Balochistan from around 1351 to 1524 CE?',
  'options': {'A': 'Samma dynasty',
   'B': 'Ghaznavid dynasty',
   'C': 'Mughal dynasty',
   'D': 'Soomra dynasty'},
  'correct': 'A'},
 '2': {'no': '2',
  'mcq': 'Who founded the Mughal Empire in 1526 after sweeping across the Khyber Pass?',
  'options': {'A': 'Sher Shah Suri',
   'B': 'Babur',
   'C': 'Akbar',
   'D': 'Jahangir'},
  'correct': 'B'},
 '3': {'no': '3',
  'mcq': 'Which architectural legacy of the Mughals is located in Lahore and is known for its magnificent structures?',
  'options': {'A': 'Shalimar Gardens',
   'B': 'Wazir Khan Mosque',
   'C': 'Badshahi Mosque',
   'D': 'Lahore Fort'},
  'correct': 'C'},
 '4': {'no': '4',
  'mcq': "Which Turkic ruler defeated the Mughal Empire in the year 1540, leading to Humayun's retreat to Kabul?",
  'options': {'A': 'Sher Shah Suri',
   'B': 'Babur',
   'C': 'Akbar',
   'D': 'Aurangzeb'},
  'correct': '

In [39]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choice":options,"Correct":correct})

In [40]:
df=pd.DataFrame(quiz_table_data)

In [41]:
df.to_csv("genaiquiz1.csv")